In [2]:
import pandas as pd
import numpy as np
pad='/usr/people/out/Documents/Finland'

trafo=pd.read_csv(f'{pad}/spreadsheettrafo.csv', delimiter=';')
trafo_icon=trafo.loc[:,'trafonr'].to_numpy()
trafo_trafo=np.zeros((len(trafo_icon),len(trafo_icon)))
cables=pd.read_csv(f'{pad}/spreadsheetcables.csv', delimiter=';')
cable_icon=cables.loc[:,'kabelnr'].to_numpy()
cablecondu=cables.loc[:,'conductivity'].to_numpy()
trafo_cable=np.zeros((len(trafo_icon),len(cable_icon)))
Z_matrix=np.zeros((len(trafo_icon),len(trafo_icon)))
Y_matrix=np.zeros((len(trafo_icon),len(trafo_icon)))
J_north=np.zeros(len(trafo_icon))
J_east=np.zeros(len(trafo_icon))

for counter,row in enumerate(trafo.loc[:,'verbonden trafo'].to_numpy()):
    row=str(row).split(',')
    for item in str(trafo.loc[counter,'alle aansluitingen']).split(','):
        trafo_cable[counter,np.where(cable_icon==int(item))]=True
    for stationnr in row:
        trafo_trafo[counter,np.where(trafo_icon==int(stationnr))]=True
for station,row in enumerate(trafo_trafo):
    Z_matrix[station,station]=trafo.loc[station,'resistance']
    for item, column in enumerate(row):
        if column:
            temp=0
            for counter,value in enumerate(trafo_cable[station,:]+trafo_cable[item,:]):
                if value==2:
                    temp+=1/cables.loc[counter,'conductivity old']
            Y_matrix[station,item]=-1/temp
            Y_matrix[station,station]+=1/temp
            
locX=np.array([58.17,97.60,106.25,120.19,125,146.63,206.73,230.29,228.85,305.29,394.23,409.13,593.75,746.15,752.40,755.77,789.9])
locY=np.array([351.44,492.79,411.06,282.69,404.33,499.52,227.40,258.65,361.06,556.25,375,543.27,452.88,441.83,520.19,244.71,256.73])

for row,line in enumerate(trafo_trafo):
    for column,item in enumerate(line):
        if item:
            J_north[row]+=-1*(locX[row]-locX[column])*Y_matrix[row,column]
            J_east[row]+=-1*(locY[row]-locY[column])*Y_matrix[row,column]

J_total=J_north+J_east            
matrix=np.identity(len(trafo_icon))+np.matmul(Y_matrix,Z_matrix)
I_GIC=np.linalg.solve(matrix,J_north)
print(I_GIC)

[-107.529829    -88.57078493  -35.73891705  -36.78435384  -16.17019924
  -26.4076676    14.89040332    8.18750803   -4.04009058  -15.3829672
   15.31618982    1.64908943   36.11180064   84.8456743    63.88921165
   29.11615701   76.61877524]


In [3]:
import pandas as pd
import numpy as np
pad='/usr/people/out/Documents/Finland'

trafo=pd.read_csv(f'{pad}/spreadsheettrafo.csv', delimiter=';')
trafo_icon=trafo.loc[:,'trafonr'].to_numpy()
trafo_trafo=np.zeros((len(trafo_icon),len(trafo_icon)))
cables=pd.read_csv(f'{pad}/spreadsheetcables.csv', delimiter=';')
cable_icon=cables.loc[:,'kabelnr'].to_numpy()
cablecondu=cables.loc[:,'conductivity'].to_numpy()
trafo_cable=np.zeros((len(trafo_icon),len(cable_icon)))
Y_matrix=np.zeros((len(trafo_icon),len(trafo_icon)))
J_north=np.zeros(len(trafo_icon))
J_east=np.zeros(len(trafo_icon))

for counter,row in enumerate(trafo.loc[:,'verbonden trafo'].to_numpy()):
    row=str(row).split(',')
    for item in str(trafo.loc[counter,'alle aansluitingen']).split(','):
        trafo_cable[counter,np.where(cable_icon==int(item))]=True
    for stationnr in row:
        trafo_trafo[counter,np.where(trafo_icon==int(stationnr))]=True
for station,row in enumerate(trafo_trafo):
    for item, column in enumerate(row):
        if column:
            temp=0
            for counter,value in enumerate(trafo_cable[station,:]+trafo_cable[item,:]):
                if value==2:
                    temp+=1/cables.loc[counter,'conductivity old']
            Y_matrix[station,item]=-1/temp
            Y_matrix[station,station]+=1/temp
    Y_matrix[station,station]+=trafo.loc[station,'conductivity']
            
locX=np.array([58.17,97.60,106.25,120.19,125,146.63,206.73,230.29,228.85,305.29,394.23,409.13,593.75,746.15,752.40,755.77,789.9])
locY=np.array([351.44,492.79,411.06,282.69,404.33,499.52,227.40,258.65,361.06,556.25,375,543.27,452.88,441.83,520.19,244.71,256.73])

for row,line in enumerate(trafo_trafo):
    for column,item in enumerate(line):
        if item:
            J_north[row]+=-1*(locX[row]-locX[column])*Y_matrix[row,column]
            J_east[row]+=-1*(locY[row]-locY[column])*Y_matrix[row,column]
            
V_GIC=np.linalg.solve(Y_matrix,J_east)
I_GIC=V_GIC*trafo.loc[:,'conductivity']
print(I_GIC)
V_GIC=np.linalg.solve(Y_matrix,J_north)
I_GIC=V_GIC*trafo.loc[:,'conductivity']
print(I_GIC)

0       4.109217
1      58.078305
2      -1.120244
3     -25.461955
4       0.688481
5      27.953407
6    -102.473163
7     -20.957842
8     -26.165158
9      79.444838
10    -18.622778
11     27.830309
12     52.546957
13     36.462691
14     68.727144
15   -105.883156
16    -55.157051
Name: conductivity, dtype: float64
0    -107.601096
1     -88.589004
2     -35.800622
3     -36.794763
4     -16.119797
5     -26.372417
6      14.926485
7       8.190802
8      -4.020773
9     -15.361470
10     15.313290
11      1.644185
12     36.147956
13     84.925932
14     63.779961
15     29.112547
16     76.618786
Name: conductivity, dtype: float64
